# 大模型

## 对话模型

### 同步调用

In [1]:
from illufly.chat import ChatOpenAI

a = ChatOpenAI()
a("你能帮我写一首关于兔子做梦的四句儿歌?", verbose=True)

小兔子做梦
梦见大胡萝卜
咬一口甜甜的
梦中笑得很开心



'小兔子做梦\n梦见大胡萝卜\n咬一口甜甜的\n梦中笑得很开心'

### 显示使用 log

In [2]:
# 使用 handler 函数的另一种方式：
from illufly.chat import ChatOpenAI
from illufly.io import log

log(ChatOpenAI(), "你能帮我写一首关于兔子做梦的四句儿歌?", verbose=True)

小兔子瞌睡梦中飘，梦见大树荡秋桥。
梦里有糖果山高插，蹦跳着找小时候乐。
醒来只是梦境皮，可笑兔子又快乐。



'小兔子瞌睡梦中飘，梦见大树荡秋桥。\n梦里有糖果山高插，蹦跳着找小时候乐。\n醒来只是梦境皮，可笑兔子又快乐。'

### 异步调用

In [1]:
# 异步输出
from illufly.chat import ChatOpenAI
from illufly.io import alog

a = ChatOpenAI(model="gpt-3.5-turbo")
await a(
    "你能帮我写一首关于兔子做梦的四句儿歌?",
    verbose=True,
    handler=alog
)
a.memory

兔子睡觉做美梦，
梦中有万花齐放的园，
小兔在花海里蹦跳，
快乐的梦境永不停。



[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant',
  'content': '兔子睡觉做美梦，\n梦中有万花齐放的园，\n小兔在花海里蹦跳，\n快乐的梦境永不停。'}]

In [2]:
# 在调用时指定模型名称
await ChatOpenAI()(
    "你能帮我写一首关于兔子做梦的四句儿歌?",
    handler=alog,
    model="gpt-3.5-turbo",
    temperature=0.1
)
a.memory

当兔子闭上眼，梦里是个仙境，
跳跃在绿草地，快乐无比轻盈。
梦中有胡萝卜，还有甜美的胡椒，
兔子笑得合不拢嘴，做梦也是如此快乐。



[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant',
  'content': '兔子睡觉做美梦，\n梦中有万花齐放的园，\n小兔在花海里蹦跳，\n快乐的梦境永不停。'}]

**在调用时指定工具，返回应当调用的工具信息**

### 工具回调

In [2]:
from illufly.chat import ChatOpenAI
from illufly.types import ToolAgent

def get_current_weather(location: str):
    """获取城市的天气情况"""
    yield f"{location}今天是晴天。 "
    
a = ChatOpenAI(tools=[ToolAgent(get_current_weather)], exec_tool=True)
a("今天广州出门要准备什么", new_chat=True, verbose=True)
a.memory

  0s [TOOLS_CALL_CHUNK] {"index": 0, "id": "call_2JXfIwLbkm7vZIjGIzn50djj", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
  0s [TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"name": "", "arguments": "{\n"}}
  0s [TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"name": "", "arguments": "\""}}
  0s [TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"name": "", "arguments": "location"}}
  0s [TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"name": "", "arguments": "\":"}}
  0s [TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"name": "", "arguments": " \""}}
  0s [TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"name": "", "arguments": "广"}}
  0s [TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"name": "", "arguments": "州"}}
  0s [TOOLS_CALL_CHUNK] {"index": 0, "id": "", "typ

[{'role': 'user', 'content': '今天广州出门要准备什么'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_2JXfIwLbkm7vZIjGIzn50djj',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{\n"location": "广州"\n}'}}]},
 {'tool_call_id': 'call_2JXfIwLbkm7vZIjGIzn50djj',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '广州今天是晴天。 '},
 {'role': 'assistant',
  'content': '今天广州是晴天，出门时可以准备以下物品：\n\n1. 太阳镜：防止阳光刺眼。\n2. 防晒霜：保护皮肤。\n3. 帽子：遮阳。\n4. 清凉舒适的衣物：适合温暖的天气。\n5. 水：保持水分。\n\n希望你今天有个愉快的出门！'}]

In [2]:
from illufly.chat import ChatOpenAI
from illufly.types import ToolAgent

def get_current_weather(location: str):
    """获取城市的天气情况"""
    yield f"{location}今天是晴天， 25摄氏度。 "

def get_prices(hours: int=1):
    """预测订羽毛球球场的价格，hours是预订球场的小时数，因为球场价格是按照小时计算的"""
    yield f"球场每小时25元，你要定{hours}小时，所以是{hours*25}元。"

a = ChatOpenAI()
toolkits = [ToolAgent(t) for t in [get_current_weather, get_prices]]
a("帮我看看广州天气，顺便看看订2小时的羽毛球球场多少钱", tools=toolkits, new_chat=True, verbose=True)
a.memory

  1s [TOOLS_CALL_CHUNK] {"index": 0, "id": "call_QTx6SGK9MSssDXd9DaKrVHt9", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
  1s [TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"name": "", "arguments": "{\"lo"}}
  1s [TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"name": "", "arguments": "catio"}}
  1s [TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"name": "", "arguments": "n\": \"广"}}
  1s [TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"name": "", "arguments": "州\"}"}}
  1s [TOOLS_CALL_CHUNK] {"index": 1, "id": "call_RxBPXPQYk7h8n1IX1H32Vcok", "type": "function", "function": {"name": "get_prices", "arguments": ""}}
  1s [TOOLS_CALL_CHUNK] {"index": 1, "id": "", "type": "function", "function": {"name": "", "arguments": "{\"ho"}}
  1s [TOOLS_CALL_CHUNK] {"index": 1, "id": "", "type": "function", "function": {"name": "", "arguments": "urs\":"}}


[{'role': 'user', 'content': '帮我看看广州天气，顺便看看订2小时的羽毛球球场多少钱'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_QTx6SGK9MSssDXd9DaKrVHt9',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'tool_call_id': 'call_QTx6SGK9MSssDXd9DaKrVHt9',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '广州今天是晴天， 25摄氏度。 '},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 1,
    'id': 'call_RxBPXPQYk7h8n1IX1H32Vcok',
    'type': 'function',
    'function': {'name': 'get_prices', 'arguments': '{"hours": "2"}'}}]},
 {'tool_call_id': 'call_RxBPXPQYk7h8n1IX1H32Vcok',
  'role': 'tool',
  'name': 'get_prices',
  'content': '球场每小时25元，你要定2小时，所以是50元。'},
 {'role': 'assistant', 'content': '广州今天天气晴朗，气温25摄氏度。如果你要订2小时的羽毛球球场，价格是50元。'}]

## Fake

### fake_llm: 模拟大模型调用过程

这可以观察提示语等内部结构。

In [1]:
from illufly.chat import FakeLLM
from illufly.io import log

log(
    FakeLLM(),
    [
        {"role": "system", "content": "你是一个写作能手。"},
        {"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}
    ],
    model="qwen2-1.5b-instruct"
)

这是一个模拟调用!



'这是一个模拟调用!'

In [2]:
FakeLLM.monitor_executors()

{'FAKE_LLM': {'max_workers': 5, 'used_workers': 0, 'waiting_threads': 0}}